In [47]:
import io
import os
import boto3
from azure.storage.blob import BlobServiceClient
import PyPDF2
import requests
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

# configure one of the following as a file source
# Local File ##########################################
file_path = 'file.pdf'
# File in Azure Blob Storage ##########################
container_name = 'content'
blob_name = 'file.pdf'
account_url = 'https://bpf7701sa.blob.core.windows.net/'
credential = DefaultAzureCredential() #'mycredential'
# File in AWS S3 #######################################
bucket_name = 'content'
object_name = 'file.csv'
aws_access_key_id = 'myaccesskey'
aws_secret_access_key = 'mysecretkey'
########################################################

text1 = read_pdf_local(file_path)
print(text)
text2 = read_pdf_azure(container_name, blob_name, account_url, credential)
print(text)
text3 = read_pdf_s3(bucket_name, object_name, aws_access_key_id, aws_secret_access_key)
print(text)

# # Call the appropriate function based on the source of the PDF
# if os.path.isfile(file_path):
#     text = read_pdf_local(file_path)
# elif account_url and credential:
#     text = read_pdf_azure(container_name, blob_name, account_url, credential)
# elif aws_access_key_id and aws_secret_access_key:
#     text = read_pdf_s3(bucket_name, object_name, aws_access_key_id, aws_secret_access_key)
#print(text)

# Function to read PDF from local file
def read_pdf_local(file_path):
    with open(file_path, 'rb') as f:
        pdf = PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to read PDF from Azure Blob Storage
def read_pdf_azure(container_name, blob_name, account_url, credential):
    blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    stream = io.BytesIO()
    blob_client.download_blob().download_to_stream(stream)
    stream.seek(0)
    pdf = PdfReader(stream)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

# Function to read PDF from AWS S3 Storage
def read_pdf_s3(bucket_name, object_name, aws_access_key_id, aws_secret_access_key):
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    obj = s3.get_object(Bucket=bucket_name, Key=object_name)
    stream = io.BytesIO(obj['Body'].read())
    stream.seek(0)
    pdf = PdfReader(stream)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text



Database, Data, BI, AI, Analytics   
Data Driven Performance Analyst  Brian P. Flynn  
Erie, CO 
brianpaulflynn@outlook.com  
last updated 1/1/2023 Product Management & Leadership  
Development, Architecture  
http://www.linkedin.com/in/brianpaulflynn  
 
OBJECTIVE  
Obtain a technical leadership position utilizing over twenty -five years of practical experience and education.  
PROFESSIONAL SUMMARY   
Growth Mindset! Architect , People &  Product Manager .  Former SQL Server expert  gone cloud . Passion 
for innovation . A data driven visionary at the intersection of business, technology and user experience. 
Possesses well rounded, sharp technical skills , balanced by strong emotional intelligence. Strategically 
progressive change agent and entrepreneurial leader who enjoys new challenges and seeks to integrate 
the sum of his experiences into one role for maximum impact . Dabbled in self -employment and 
discovered a passion for Product Management. Interested in the burgeoning fiel

HttpResponseError: This request is not authorized to perform this operation using this permission.
RequestId:f79162a5-701e-0032-6080-622d8c000000
Time:2023-03-29T20:52:14.0452804Z
ErrorCode:AuthorizationPermissionMismatch
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationPermissionMismatch</Code><Message>This request is not authorized to perform this operation using this permission.
RequestId:f79162a5-701e-0032-6080-622d8c000000
Time:2023-03-29T20:52:14.0452804Z</Message></Error>